In [ ]:
#Creating the Memory class with methods for storing, loading, printing, and selecting from memory.
class Memory:

    def __init__(self):
        #This is the memory bank that holds all addresses and values in main memory
        self.memory_bank = {}
        for i in range(0, 256, 4):
            self.memory_bank[i] = None
        self.cache_is_active = False

    def store_into_memory(self, address, data):
        self.memory_bank[address] = data
        #This stores values into an address in main memory

    def load_from_memory(self, address):
        return self.memory_bank[address]
        #This returns a specified value from a memory address.

    def print_memory(self):
        for address, data in self.memory_bank.items():
            print (f"{address}:{data}")

    def select_from_available_memory(self):
        for i in self.memory_bank:
            if self.memory_bank[i] == None:
                return i
            else:
                continue
        return ("No available slots left in memory.")
    
#Adds the isntruction to the list as an instance of the Instruction class, and selects an available memory slot to load it into.
    def load_instr_to_memory(self, file):
        list_of_instructions = []
        with open(file) as instruction_file:
            list_of_instructions += instruction_file.readlines()
        for instruction in list_of_instructions:
            selected_instruction = Instruction(instruction)
            self.store_into_memory(self.select_from_available_memory, selected_instruction)

In [1]:
#Creating the CacheEntry class which has tag and value as attributes.
class CacheEntry:
    def __init__(self, tag, value):
        self.tag = tag
        self.value = value
# The __repr__ method will print out the tag and value of a cache entry
    def __repr__(self):
        return (f"{self.tag}:{self.value}")


#Creating the Cache class with cache_bank, size, and fifo_index attributes.
class Cache:

    def __init__(self, cache_size = 4):
        self.cache_bank = []
        self.cache_size = cache_size
        self.fifo_index = 0

#Methods attached to the cache class include storing into cache, loading from cache, replacing an entry in cache, and printing the cache.

    def store_into_cache(self, tag, value, memory):
        for entry in self.cache_bank:
            if entry.tag == tag:
                print("There is already an entry with this tag in the cache, updating value of entry with new value")
                entry.value = value
                return
        if len(self.cache_bank) == self.cache_size:
            print("Cache is full, implementing replacement policy")
            #If the cache bank is full, it will perform the replace entry method.
            self.replace_entry(tag, value, memory)
        else:
            self.cache_bank.append(CacheEntry(tag, value))

#Loading from cache based on a cache hit.
    def load_from_cache(self, tag):
        for entry in self.cache_bank:
            if entry.tag == tag:
                print("Cache hit, returning value")
                return entry.value
        return "No such entry with corresponding tag in cache"

#Inputs old value into main memory before replacing the value.
    def replace_entry(self, tag, value, memory):
        current_entry = self.cache_bank[self.fifo_index]
        #inputting into main memory before replacement
        memory.store_into_memory(current_entry.tag, current_entry.value)
        self.cache_bank[self.fifo_index].tag = tag
        self.cache_bank[self.fifo_index].value = value
        #Cache entry has been replaced
        if self.fifo_index > self.cache_size:
            self.fifo_index = 0
            #Replacement index reset to 0.
#Method for printing all entries in cache.      
    def print_cache(self):
        for entry in self.cache_bank:
            print(entry)

In [ ]:
#Creating the Instruction class. It has an attribute for each part of the instruction. Attributes inclide the destination register /
#... the source register, the offset, opcode, constant, address, cache_bool, and deciphering the instruction
class Instruction:

    def __init__(self, instruction_string):
        self.instruction_string = instruction_string
        self.rd = None
        self.rs = None
        self.rt = None
        self.offset = None
        self.opcode = None
        self.constant = None
        self.address = None
        self.cache_bool = None
        self.decipher_instruction(instruction_string)
    def __repr__(self):
        stripped_instruction = self.instruction_string.strip("\n")
        return stripped_instruction

#Decipher instruction will break down the instruction into its parts. It first interprets the opcode. Based on the opcode, it knows which parts correspond to what.
#It goes through a series of elif statements to see which opcode is matched.
    def decipher_instruction(self, instruction_string):
        split_instruction = instruction_string.split(',')
        arithmetic_operands = ['AND', 'SUB', 'SLT', 'MULT', 'AND', 'OR']
        constant_operands = ['ADDI', 'ORI', 'ANDI']
        jump_operands = ['J', 'JAL']
#The above are lists of the different possible opcodes grouped into operand classes.
#The below takes the first part of the split instruction to get the opcode.
        self.opcode = split_instruction[0]
        #If the opcode is arithmetic, the format is as follows.
        if self.opcode in arithmetic_operands:
            #The below line in each if block makes sure the length of the split instruction is correct.
            if len(split_instruction) != 4:
                raise ValueError("Invalid instruction length")
            try:
                #Trying to assign the parts of the instruction to the corresponding instance variables of the instruction.
                self.rd = int(split_instruction[1])
                self.rs = int(split_instruction[2])
                self.rt = int(split_instruction[3])
            except:
                raise ValueError("Invalid instruction type, values following opcode must be of type int")
        elif self.opcode in constant_operands:
            if len(split_instruction) != 4:
                raise ValueError("Invalid instruction length")  
            try:
                self.rd = int(split_instruction[1])
                self.rs = int(split_instruction[2])
                self.constant = int(split_instruction[3])
            except:
                raise ValueError("Invalid instruction type, values following opcode must be of type int")
        elif self.opcode in jump_operands:
            if len(split_instruction) != 2:
                raise ValueError("Invalid instruction length")
            try:
                self.address = int(split_instruction[1])
            except:
                raise ValueError("Invalid instruction type, values following opcode must be of type int")
        elif self.opcode == 'CACHE':
            if len(split_instruction) != 2:
                raise ValueError("Invalid instruction length") 
            try:
                self.cache_bool = int(split_instruction[1])
            except:
                raise ValueError("Invalid instruction type, values following opcode must be of type int")
        elif self.opcode == 'BNE' or self.opcode == 'BQE':
            if len(split_instruction) != 4:
                raise ValueError("Invalid instruction length")
            try:
                self.rs = int(split_instruction[1])
                self.rt = int(split_instruction[2])
                self.offset = int(split_instruction[3])
            except:
                raise ValueError("Invalid instruction type, values following opcode must be of type int")
        elif self.opcode == 'LW':
            if len(split_instruction) != 3:
                raise ValueError("Invalid instruction length")
            self.rd = int(split_instruction[1])
            try:
                if '(' in split_instruction:
                    split_mem_address = split_instruction[-1].split('(')
                    split_mem_address_stripped = split_mem_address[-1].strip('()')
                    self.rs = int(split_mem_address_stripped)
                    self.offset = int(split_mem_address[0])
                else:
                    self.rs = split_instruction[-1]
            except:
                ValueError("Invalid instruction type, values following opcode must be of type int")
        elif self.opcode == 'SW':
            if len(split_instruction) != 3:
                raise ValueError("Invalid instruction length")
            try:
                self.rs = int(split_instruction[1])
                if '(' in split_instruction:
                    split_mem_address = split_instruction[-1].split('(')
                    split_mem_address_stripped = split_mem_address[-1].strip('()')
                    self.rt = int(split_mem_address_stripped)
                    self.offset = int(split_mem_address[0])
                else:
                    self.rt = split_instruction[-1]
            except:
                ValueError("Invalid instruction type, values following opcode must be of type int")
        elif self.opcode == 'JR':
            if len(split_instruction) != 2:
                raise ValueError("Invalid instruction length")
            try:
                self.address = int(split_instruction[-1])
            except:
                raise ValueError("Invalid instruction type, values following opcode must be of type int")
            if split_instruction[-1] != 31:
                raise ValueError("Invalid instruction, address of JR instruction should be register $31")
        else:
            raise ValueError('Invalid Opcode')